In [2]:
# Import the packages we'll use

import numpy as np
import os, glob, csv

import tensorflow as tf
# # librosa is a widely-used audio processing library
import librosa

import sklearn
import os
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset
from torchaudio import transforms as t
import pandas as pd
import torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as nnF

# # for plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
os.getcwd()

'/Users/zayyanmatin/Desktop/Masters/DLAM'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
os.listdir()

['non project stuff',
 'Pytorch_Audio_Classifier_Procedure_Raw-Copy1.ipynb',
 'Pytorch_Audio_Classifier_Procedure-Testing.ipynb',
 'Pytorch_Audio_Classifier_Procedure_Raw-Copy3.ipynb',
 'Untitled1.ipynb',
 'test1.rtf',
 'Pytorch_Audio_Classifier_Procedure_Mel-Copy1.ipynb',
 '.DS_Store',
 'mel.pt',
 'bgBinary.csv',
 'Pytorch_Audio_Classifier_Procedure_Mel-Copy3.ipynb',
 'Pytorch_Audio_Classifier_Procedure_Raw-Copy2.ipynb',
 'Untitled.ipynb',
 'bird_bg04.wav',
 'bg_class.csv',
 'Pytorch_Audio_Classifier_Procedure_Mel-Copy2.ipynb',
 'bgBinary 2.numbers',
 'other projects',
 'bgBinary_Crowd.csv',
 'Training_Validation_Maker.ipynb',
 'bgBinary.numbers',
 'env',
 'Pytorch_Audio_Classifier_Procedure_Fusion.ipynb',
 'WaveNet-master ',
 'raw.pt',
 '.ipynb_checkpoints',
 'tensorflow-wavenet-master',
 'Isolated_urban_sound_database',
 'Pytorch_Audio_Classifier_Procedure_Raw.ipynb',
 'Pytorch_Audio_Classifier_Procedure_Mel.ipynb']

In [6]:
csvData = pd.read_csv('./bgBinary_Crowd.csv')
print(len(csvData))
print(csvData.iloc[80,2])

162
0


In [18]:
class UrbanSoundDataset(Dataset):

    
    def __init__(self, csv_path, file_path, folderList):
        csvData = pd.read_csv(csv_path)
        #initialize lists to hold file names, labels, and folder numbers
        self.file_names = []
        self.labels = []
        self.folders = []
        #loop through the csv entries and only add entries from folders in the folder list
        for i in range(0,len(csvData)):
            if csvData.iloc[i, 3] in folderList:
                self.file_names.append(csvData.iloc[i, 0])
                self.labels.append(csvData.iloc[i, 2])
                self.folders.append(csvData.iloc[i, 3])
                
        self.file_path = file_path
        self.mixer = torch.mean #Two channels will convert to one
        self.folderList = folderList
        
    def __getitem__(self, index):
        #format the file path and load the file
        path = self.file_path + "fold" + str(self.folders[index]) + "/" + self.file_names[index]
        
        #load returns a tensor with the sound data and the sampling frequency (44.1kHz)
        sound = torchaudio.load(path, out = None, normalization = True)

        soundData = torch.mean(sound[0], 0, keepdim=False, out=None)
        
        
        soundData = soundData.unsqueeze_(-1)

        
        #downsample the audio to ~8kHz
        tempData = torch.zeros([160000, 1]) #tempData accounts for audio clips that are too short
        
        if soundData.numel() < 160000:
            tempData[:soundData.numel()] = soundData[:]
        else:
            tempData[:] = soundData[:160000]
        
        soundData = tempData
        
        soundFormatted = torch.zeros([32000, 1])
        soundFormatted[:32000] = soundData[::5] #take every fifth sample of soundData
        soundFormatted = soundFormatted.squeeze(-1)
        soundMel = torchaudio.transforms.MFCC(sound[1]/5)(soundFormatted)
        soundMel = soundMel.unsqueeze_(0)
#         soundMel = soundMel.permute(1, 0,2)

        
        
        return soundMel, self.labels[index]
    
    def __len__(self):
        return len(self.file_names)

    
csv_path = './bgBinary_Crowd.csv'
file_path = './Isolated_urban_sound_database/'

train_set = UrbanSoundDataset(csv_path, file_path, range(0,4))
test_set = UrbanSoundDataset(csv_path, file_path, [4])
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 32, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 32, shuffle = True, **kwargs)

Train set size: 130
Test set size: 32


In [19]:
class CnnAudioNet(nn.Module):
    def __init__(self):
        super(CnnAudioNet,self).__init__()

        self.Fc_features = 40
        self.C1 = nn.Conv2d(1,32,3,padding=1)
        self.C2 = nn.Conv2d(32,64,3,padding=1)
        self.C3 = nn.Conv2d(64,64,3,padding=1)
        self.C4 = nn.Conv2d(64,64,3,padding=1)
        self.C5 = nn.Conv2d(64,64,3,padding=1)
        
        self.BN1 = nn.BatchNorm2d(32)
        self.BN2 = nn.BatchNorm2d(64)
        self.BN3 = nn.BatchNorm2d(64)
        self.maxpool1 = nn.MaxPool2d(2,2)
        self.maxpool2 = nn.MaxPool2d((1,2),(1,2))
        
        
        self.fc1 = nn.Linear(64*1*5,128)
        self.fc2 = nn.Linear(128,10)
        self.dropout = nn.Dropout(0.25)
        self.Bat1 = nn.BatchNorm1d(128)

        
        
    def forward(self,x):
        # add sequence of convolutional and max pooling layers
        x = self.C1(x)
        x = self.BN1(x)
        x = self.maxpool1(x)
        x = self.maxpool1(F.relu(self.BN2(self.C2(x))))
        x = self.maxpool1(F.relu(self.BN2(self.C3(x))))
        x = self.maxpool1(F.relu(self.BN3(self.C4(x))))
        x = self.maxpool1(F.relu(self.BN3(self.C5(x))))
        # flatten image input
        x =x.permute(0,2,1,3)
        x=torch.flatten(x, start_dim=2, end_dim=3)
        # add dropout layer
        x =  self.dropout(self.fc1(x))
        print("x" + str(x.shape))
        x = F.log_softmax(self.fc2(x),  dim =2)
        print("x2" + str(x.shape))

        return x
        
model = CnnAudioNet()
model.to(device)
print(model)

CnnAudioNet(
  (C1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (C2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (C3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (C4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (C5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (BN1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (maxpool2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=Fal

In [14]:
optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.0001)

In [15]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
        output = output.permute(1, 0, 2)
#         output = output.unsqueeze(1)
#         print("LossOutPre" + str(output.shape))
#         print("LossOut" + str(output[0].shape))
#         print("LossOut" + str(output[0][1]))
        loss = F.nll_loss(output[0], target) #the loss functions expects a batchSizex10 input
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

        
    if loss <= 0.00010:
        
        return 1
    
    else:
        return 0

In [16]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        output = output.permute(1, 0, 2)
        pred = output.max(2)[1] # get the index of the max log-probability
        correct += pred.eq(target).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:
log_interval = 2
for epoch in range(1, 27):
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
    
    train(model, epoch)
    test(model, epoch)

    

xtorch.Size([32, 1, 128])
x2torch.Size([32, 1, 10])
Train Epoch: 1 [0/130 (0%)]	Loss: 2.027590
xtorch.Size([32, 1, 128])
x2torch.Size([32, 1, 10])
xtorch.Size([32, 1, 128])
x2torch.Size([32, 1, 10])
Train Epoch: 1 [64/130 (40%)]	Loss: 0.216070


KeyboardInterrupt: 

In [15]:
torch.save(model.state_dict(), 'mel.pt')